In [ ]:
!pip install qutip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 15.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
import builtins
from typing import List

# Set Up + Useful Functions

In [ ]:
zero_ket = basis(2,0)
zero_rho = zero_ket * zero_ket.dag()
one_rho = basis(2,1) * basis(2,1).dag()
I = lambda: qeye(2)


def initial_rho(num_qubits):
  return tensor([zero_rho for i in range (num_qubits)])


def bell_state(state):
  """
  Returns the requested Bell state as a density matrix.

  Parameters:
  state (str): The label of the Bell state to return.
                Must be one of 'phi+', 'phi-', 'psi+', or 'psi-'.

  Returns:
  Qobj: The density matrix of the requested Bell state.
  """
  if state == 'phi+':
      phi_plus_ket = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_plus_ket * phi_plus_ket.dag()

  elif state == 'phi-':
      phi_minus_ket = (tensor(basis(2, 0), basis(2, 0)) - tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_minus_ket * phi_minus_ket.dag()

  elif state == 'psi+':
      psi_plus_ket = (tensor(basis(2, 0), basis(2, 1)) + tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_plus_ket * psi_plus_ket.dag()

  elif state == 'psi-':
      psi_minus_ket = (tensor(basis(2, 0), basis(2, 1)) - tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_minus_ket * psi_minus_ket.dag()

  else:
      raise ValueError("Invalid Bell state label")


def pad_op(op, left_padding, right_padding):
    """
    Pads the given operator with identity operators on the left and right.

    Parameters:
    op (Qobj): The operator to be padded.
    left_padding (int): The number of identity operators to pad on the left.
    right_padding (int): The number of identity operators to pad on the right.

    Returns:
    Qobj: The padded operator.
    """

    if left_padding > 0:
        left_op = tensor([I()] * left_padding)
        op = tensor(left_op, op)

    if right_padding > 0:
        right_op = tensor([I()] * right_padding)
        op = tensor(op, right_op)

    return op







def state_inserter(rho_initial, target_qubits, state):
  """
    Takes a density matrix of a network, inserts the state 'state' into the slots of the target_qubits.
    Assumes that target qubits are beside eachother and that no entanglement exists between
    qubits at either side of the target_qubits.

    Returns full density matrix of updated network
  """
  if isinstance(target_qubits, int):
    num_qubits_left = target_qubits
    num_qubits_right = num_qubits - target_qubits - 1

    qubits_left_list = [i for i in range(num_qubits_left)]
    qubits_right_list = [i + num_qubits_left + 1 for i in range(num_qubits_right)]

  elif isinstance(target_qubits, builtins.list):
    num_qubits_left = target_qubits[0]
    num_qubits_right = num_qubits - target_qubits[-1] - 1

    qubits_left_list = [i for i in range(num_qubits_left)]
    qubits_right_list = [i + num_qubits_left + len(target_qubits) for i in range(num_qubits_right)]

  else:
    raise TypeError('Invalid data type for target_qubit, must be int or list')

  rho = state
  if len(qubits_left_list) != 0:
    rho_qubits_left = rho_initial.ptrace(qubits_left_list)
    rho = tensor(rho_qubits_left, rho)

  if len(qubits_right_list) != 0:
    rho_qubits_right = rho_initial.ptrace(qubits_right_list)
    rho = tensor(rho, rho_qubits_right)

  return rho


def time_for_link(eta_eff, T_p):
  """
  Gets the time taken to generate entanglement between 2 adjacent nodes
  Does this by generating random samples from a geometric distribution with
  success probability eta_eff and then multiplying by time taken for one trial.

  Parameters:
  eta_eff (float): Success probability for the geometric distribution.
  num_samples (int): Number of random samples to generate.

  Returns:
  np.ndarray: Array of random samples from the geometric distribution.
  """
  no_of_trials = np.random.geometric(eta_eff)
  time_per_trial = T_p + 2 * d / c
  time_per_link = no_of_trials * time_per_trial
  return time_per_link


def dephasing_channel(rho, t, left_padding, right_padding):
  """
  Applies the dephasing channel to a given density matrix `rho`.

  Parameters:
  rho (Qobj): The initial density matrix of the quantum system.
  t (float): The time over which the dephasing occurs.
  left_padding (int): The number of identity operators to pad on the left.
  right_padding (int): The number of identity operators to pad on the right.
  T_dp (float): The dephasing time constant.

  Returns:
  Qobj: The density matrix of the quantum system after the dephasing channel has been applied.
  """
  dp_prob = (1 - np.exp(-t / T_dp)) / 2
  Z_op = np.sqrt(dp_prob) * pad_op(op=sigmaz(), left_padding=left_padding, right_padding=right_padding)
  I_op = np.sqrt(1 - dp_prob) * pad_op(op=I(), left_padding=left_padding, right_padding=right_padding)
  rho_t = I_op * rho * I_op.dag() + Z_op * rho * Z_op.dag()
  return rho_t



def dark_counts(rho_initial, target, P_link):
  num_qubits_left = target
  num_qubits_right = (num_qubits -1) - target
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]

  # channel efficiency
  n_ch = lambda L: np.exp(-L/L_att)

  # total probability that a pair is established
  n = P_link * n_ch(d)

  # the chance for a detector to click (including dark counts)
  n_eff = 1 - (1 - n) * ((1 - p_d) ** 2)

  # given a click occurs, the probability it is from a real event
  alpha = lambda n: (n * (1 - p_d)) / n_eff
  k = 1 - alpha(n)

  # defining Krauss operators
  K0 = np.sqrt(1 - 3 * k / 4) * pad_op(op=I(), left_padding=num_qubits_left, right_padding=num_qubits_right)
  K1 = np.sqrt(k / 4) * pad_op(op=sigmax(), left_padding=num_qubits_left, right_padding=num_qubits_right)
  K2 = np.sqrt(k / 4) * pad_op(op=sigmay(), left_padding=num_qubits_left, right_padding=num_qubits_right)
  K3 = np.sqrt(k / 4) * pad_op(op=sigmaz(), left_padding=num_qubits_left, right_padding=num_qubits_right)
  error_ops = [K1, K2, K3]

  # applying Krauss ops
  rho = K0 * rho_initial * K0.dag()
  for op in error_ops:
    rho += op * rho_initial * op.dag()
  return rho



def initial_F_rho(rho_initial, target_qubits: List[int], F_initial: float):
  rho_bell = F_initial * bell_state('phi+') + ((1 - F_initial) / 3) * (bell_state('phi-') +
                                                                    bell_state('psi+') +
                                                                    bell_state('psi-'))
  rho = state_inserter(rho_initial=rho_initial, target_qubits=target_qubits, state=rho_bell)
  return rho

def create_qubit_time_lists(n):
  """
  Creates a dictionary with keys for qubit time lists.

  Parameters:
  n (int): The number of qubits.

  Returns:
  dict: A dictionary where each key is a string in the format 'q{i}_time' (where {i} is the qubit index)
        and the value is initialized to 0.

  Example:
  >>> create_qubit_time_lists(3)
  {'q0_time': 0, 'q1_time': 0, 'q2_time': 0}
  """
  lists_dict = {}
  for i in range(n):
      lists_dict[f'q{i}_time'] = 0
  return lists_dict

# Entanglement Generation and Swapping

In [70]:
def entanglement_generation(rho_initial, target_qubits, F_initial, n, P_link, T_p, eta_eff):
  """
  Generates entanglement across 2 neighboring repeater stations q1 and q2 by
  creating a phi+ Bell state density matrix with fidelity F_initial and inserting
  it into rho_initial. This function assumes no pre-existing entanglement across
  the selected qubits.

  Parameters:
  rho_initial (Qobj): Initial state of the quantum system.
  target_qubits (list of int): Indices of the target qubits to be entangled.
  F_initial (float): Initial fidelity of the phi+ Bell state.
  num_qubits (int): Total number of qubits in the system.
  n (int): Number of trials or attempts.
  P_link (float): Probability of successful entanglement.
  T_p (float): Time period for the process.
  eta_eff (float): Efficiency factor.

  Returns:
  Qobj: The density matrix of the system after entanglement generation.
  """
  # SET UP
  # getting the number of qubits either side of the pair and making a list of them
  q0, q1 = target_qubits[0], target_qubits[1]
  num_qubits_left = q0
  num_qubits_right = (num_qubits - 1) - q1
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]


  # GENERATING PHI+ BELL PAIR WITH FIDELITY F_INITIAL
  rho = initial_F_rho(rho_initial, [q0, q1], F_initial)

  # ADDING MEMORY NOISE
  # dephasing channel for q0 (stays in memory for twice as long)
  rho = dephasing_channel(rho=rho, t = 2 * d/c, left_padding = num_qubits_left, right_padding = num_qubits_right + 1)

  # dephasing channel for q1
  rho = dephasing_channel(rho=rho, t = d/c, left_padding = num_qubits_left + 1, right_padding = num_qubits_right)

  # ADDING DARK COUNTS NOISE
  rho = dark_counts(rho_initial=rho, target=q1, P_link=P_link)
  #rho = dark_counts_entang_gen(rho_initial=rho,q0=q0, q1=q1, P_link=P_link)
  #TIMING
  global current_time
  time_entang = time_for_link(eta_eff, T_p)
  #print('entang time', current_time)

  current_time += time_entang
  globals()[f'q{q0}_time'] = current_time
  globals()[f'q{q1}_time'] = current_time

  return rho

In [71]:
def entanglement_swapping(rho_initial, q0q1, q2q3, lambda_BSM):
  """
  rho_initial = denisty matrix
  q0q1 = list of length 2
  q2q3 = list of length 2
  lambda_BSM = float [0,1], bell state measurement ideality
  """
  # SET UP
  q0, q1 = q0q1[0], q0q1[1]
  q2, q3 = q2q3[0], q2q3[1]
  num_qubits_left = q0
  num_qubits_right = (num_qubits - 1) - q3
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]
  seperation = (q3 - q0) - 1
  bell_measure_op = tensor(I(), bell_state('phi+'), I())
  qubits = [q0, q1, q2, q3]

  # getting the initial state of the 4 qubits and creating 4 qubit density matrix rho
  rho_q0q1 = rho_initial.ptrace(q0q1)
  rho_q2q3 = rho_initial.ptrace(q2q3)
  rho = tensor(rho_q0q1, rho_q2q3)


  # DEPHASING SINCE LAST OPERATION
  # getting  the current time and the times that each of the qubits were last updated
  global current_time
  q0_time = globals()[f'q{q0}_time']
  q1_time = globals()[f'q{q1}_time']
  q2_time = globals()[f'q{q2}_time']
  q3_time = globals()[f'q{q3}_time']

  # applying dephasing memory error for the time that has passed since their state was updated
  for i, qubit in enumerate(qubits):
    rho = dephasing_channel(rho=rho, t = (current_time - globals()[f'q{qubit}_time']), left_padding = i, right_padding = (3-i))


  # PERFORMING BELL MEASUREMENT
  # project 1, 2 into phi+ bell state and take the ptrace of qubits 0, 3
  # rho is now a 2 qubit density matrix
  rho = (bell_measure_op * rho * bell_measure_op.dag()).ptrace([0,3]).unit()


  # ADDING BSM ERROR
  rho = lambda_BSM * rho + (1 - lambda_BSM) / 4 * tensor(I(), I())


  # PLACING QUBITS 0,3 BACK INTO RHO OF WHOLE SYSTEM
  # adding qubits to left of state
  if len(qubits_left_list) != 0:
    rho_qubits_left = rho_initial.ptrace(qubits_left_list)
    rho = tensor(rho_qubits_left, rho)

  # adding the qubits between the newly entangled qubits q0, q3 to the right
  rho_middle = tensor([zero_rho for i in range(seperation)])
  rho = tensor(rho, rho_middle)

  # swapping q3 to the end of the tensor product so that now rho_middle is indeed in the middle
  perm_order = [i for i in range(q3+1)]
  q3_index = perm_order.pop(num_qubits_left+1) # remove the index where q3 is now from list (where q1 normally is)
  perm_order.append(q3_index) # append q3 index to end of list, moving q3 to the end
  rho = rho.permute(perm_order)

  # adding qubits to the right of state
  if len(qubits_right_list) != 0:
    rho_qubits_right = rho_initial.ptrace(qubits_right_list)
    rho = tensor(rho, rho_qubits_right)


  # UPDATING TIMING
  # assuming instantaneous BSM
  globals()[f'q{q0}_time'] = current_time
  globals()[f'q{q1}_time'] = current_time
  globals()[f'q{q2}_time'] = current_time
  globals()[f'q{q3}_time'] = current_time
  #print(current_time)
  return rho

# Defining Global Parameters

In [76]:
# DEFINING GLOBAL PARAMETERS
# Geometric Set Up
num_qubits = 8
total_L = 100e3
d = total_L/(num_qubits - 1)

# Optical Fiber Properties
L_att = 22e3       # attenuation length of optical fiber
c = 2e8            # speed of light in fiber optic

# Global Error Parameters
T_dp = 100         # dephasing time constant (memory error)
p_d = 0            # probability a dark count will occur in a detection window

# DEFINING ENTANGLEMENT GENERATION PARAMETERS
entang_gen_args = {
    'n' : 1,             # probability a link is established between 2 neighbouring repeater stations
    'P_link' : 1,        # probability a link is established without considering distance based losses
    'T_p' : 0,           # time to prepare an entangled pair
    'eta_eff' : 1        # effective channel efficiency
}

# Example Usage

In [77]:
# simulating an 8 node network, entangling the 2 end nodes
current_time = 0
qubit_times = create_qubit_time_lists(num_qubits)

rho = initial_rho(num_qubits)
rho = entanglement_generation(rho_initial=rho, target_qubits=[0,1], F_initial = 1, **entang_gen_args)
rho = entanglement_generation(rho_initial = rho, target_qubits=[2,3], F_initial = 1, **entang_gen_args)
rho = entanglement_generation(rho_initial = rho, target_qubits=[4,5], F_initial = 1, **entang_gen_args)
rho = entanglement_generation(rho_initial = rho, target_qubits=[6,7], F_initial = 1, **entang_gen_args)
rho = entanglement_swapping(rho_initial = rho, q0q1 = [0,1], q2q3=[2,3], lambda_BSM= 1)
rho = entanglement_swapping(rho_initial = rho, q0q1 = [4,5], q2q3=[6,7], lambda_BSM= 1)
rho = entanglement_swapping(rho_initial = rho, q0q1 = [0,3], q2q3=[4,7], lambda_BSM= 1)

# PRINTING OUT PARTIAL TRACE OF ALL QUBITS
print('PARTIAL TRACE OF ALL QUBITS')
for i in range(num_qubits):
  print(f'Qubit{i}\n', np.round(rho.ptrace(i).full(),3), '\n')

# PRINTING OUT 2 QUBIT DENSITY MATRIX OF ENTANGLED STATE
entangled_state = rho.ptrace([0,num_qubits-1])
print('STATE OF ENTANGLED END NODES \n',np.round(entangled_state.full(), 3))

# TESTING FIDELITY
#phi_plus_ket = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
#print('\nFidelity:', np.round(phi_plus_ket.dag() * state_03 * phi_plus_ket, 3))


PARTIAL TRACE OF ALL QUBITS
Qubit0
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

Qubit1
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit2
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit3
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit4
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit5
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit6
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

Qubit7
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 

STATE OF ENTANGLED END NODES 
 [[0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]]


# Unit Tests

In [74]:
import unittest
from qutip import qeye, tensor

# Define the I function as per qutip
I = lambda: qeye(2)

# The function to be tested
def pad_op(op, left_padding, right_padding):
    if left_padding > 0:
        left_op = I()
        for i in range(left_padding - 1):
            left_op = tensor(left_op, I())
        op = tensor(left_op, op)

    if right_padding > 0:
        right_op = I()
        for i in range(right_padding - 1):
            right_op = tensor(right_op, I())
        return tensor(op, right_op)
    else:
        return op

class TestPadOp(unittest.TestCase):
    def test_no_padding(self):
        op = qeye(2)  # Identity operator of size 2
        result = pad_op(op, 0, 0)
        self.assertTrue(result == op)

    def test_left_padding_only(self):
        op = qeye(2)  # Identity operator of size 2
        result = pad_op(op, 2, 0)
        expected = tensor(I(), I(), op)
        self.assertTrue(result == expected)

    def test_right_padding_only(self):
        op = qeye(2)  # Identity operator of size 2
        result = pad_op(op, 0, 2)
        expected = tensor(op, I(), I())
        self.assertTrue(result == expected)

    def test_both_padding(self):
        op = qeye(2)  # Identity operator of size 2
        result = pad_op(op, 1, 1)
        expected = tensor(I(), op, I())
        self.assertTrue(result == expected)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


....
----------------------------------------------------------------------
Ran 4 tests in 0.011s

OK


# Testing global variables

In [ ]:
def create_global_variables(n):
    for i in range(n):
        globals()[f'q{i}_time'] = 0

# Define a function to update global variables
def update_global_variables(n):
    for i in range(n):
        globals()[f'q{i}_time'] = (i * 10)

In [ ]:
create_global_variables(5)
update_global_variables(5)
for i in range(5):
    print(globals()[f'q{i}_time'])

0
10
20
30
40


In [ ]:
# Define a function to change q{n}_time
def change_q_time(n, new_value):
    globals()[f'q{n}_time'] = new_value

# Example usage:
create_global_variables(5)
update_global_variables(5)

print("Before change:")
for i in range(5):
    print(globals()[f'q{i}_time'])  # This should print the values 0, 10, 20, 30, 40

change_q_time(1, 100)

print("After change:")
for i in range(5):
    print(globals()[f'q{i}_time'])  # This should print the values 0, 100, 20, 30, 40


Before change:
0
10
20
30
40
After change:
0
100
20
30
40
